In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 8e14e0e6-b9ec-4b48-9528-c5d14553be2a, 3, Finished, Available, Finished)

In [2]:
%%sql
create table if not exists LakeHouse_2.gold_shipmode
(
    ShipMode_ID	Long	,
Ship_Mode	string	,
Created_TS	timestamp,
Modified_TS timestamp
)

StatementMeta(, 8e14e0e6-b9ec-4b48-9528-c5d14553be2a, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from LakeHouse_2.gold_shipmode").first()[0]

StatementMeta(, 8e14e0e6-b9ec-4b48-9528-c5d14553be2a, 5, Finished, Available, Finished)

In [4]:
Max_Date

StatementMeta(, 8e14e0e6-b9ec-4b48-9528-c5d14553be2a, 6, Finished, Available, Finished)

'1900-01-01'

In [5]:
df_bronze=spark.sql("select distinct Ship_Mode from LakeHouse_2.bronze_sales where Modified_TS>'{}'".format(Max_Date))

StatementMeta(, 8e14e0e6-b9ec-4b48-9528-c5d14553be2a, 7, Finished, Available, Finished)

In [6]:
Max_ID=spark.sql("select coalesce(max(ShipMode_ID),0) from LakeHouse_2.gold_shipmode").first()[0]

StatementMeta(, 8e14e0e6-b9ec-4b48-9528-c5d14553be2a, 8, Finished, Available, Finished)

In [7]:
df_final=df_bronze.withColumn("ShipMode_ID",monotonically_increasing_id()+Max_ID+1)
df_final.createOrReplaceTempView("ViewShipMode")

StatementMeta(, 8e14e0e6-b9ec-4b48-9528-c5d14553be2a, 9, Finished, Available, Finished)

In [8]:
%%sql
merge into LakeHouse_2.gold_shipmode as gs 
using ViewShipMode as vs  
on gs.Ship_Mode=vs.Ship_Mode
when MATCHED then 
update set
Modified_TS= current_timestamp()

when not matched then
INSERT
(
gs.ShipMode_ID,
gs.Ship_Mode,
gs.Created_TS,
gs.Modified_TS
)
VALUES
(
vs.ShipMode_ID,
vs.Ship_Mode,
current_timestamp(),
current_timestamp()

)

StatementMeta(, 8e14e0e6-b9ec-4b48-9528-c5d14553be2a, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>